In [1]:
!pip install pytorch-transformers

In [2]:
!pip install transformers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Final_project')

In [5]:
import os
os.listdir('/content/drive/MyDrive/Final_project')

['bert_model.pkl',
 'Bart_v2.ipynb',
 'Final (1).ipynb',
 '.ipynb_checkpoints',
 'bert_emotional_classification.ipynb',
 'Parser.ipynb',
 'Parser.py',
 '__pycache__',
 'Final.ipynb']

In [6]:
from Parser import get_reviews_on_page, get_reviews

godzilla_vs_kong
reviews from page [1] have been added
reviews from page [2] have been added
reviews from page [3] have been added
reviews from page [4] have been added
reviews from page [5] have been added
reviews from page [6] have been added
reviews from page [7] have been added
reviews from page [8] have been added
reviews from page [9] have been added
reviews from page [10] have been added


In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import requests
import json
import re
import os

import pickle

try:
    import transformers
    from transformers import BartTokenizer, BartForConditionalGeneration
except ImportError:
  print("Error")
from IPython.display import display, Markdown

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [9]:
def modified_texts(texts):
    texts = ["[CLS] " + str(text) + " [SEP]" for text in texts]    
    
    tokenized_texts = [tokenizer.tokenize(sent[:512]) for sent in texts]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    input_ids = pad_sequences(
        input_ids,
        maxlen=100,
        dtype="long",
        truncating="post",
        padding="post"
    )
    
    attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

    classification_inputs = torch.tensor(input_ids).to(torch.int64)
    classification_masks = torch.tensor(attention_masks).to(torch.int64)

    classification_data = TensorDataset(
        classification_inputs,
        classification_masks
    )

    classification_dataloader = DataLoader(
        classification_data, 
        sampler=SequentialSampler(classification_data),
        batch_size=32
    )    
    
    return classification_dataloader

In [10]:
url = "https://www.rottentomatoes.com/m/zack_snyders_justice_league/reviews?type=user"
reviews = get_reviews(url)
reviews

zack_snyders_justice_league
reviews from page [1] have been added
reviews from page [2] have been added
reviews from page [3] have been added
reviews from page [4] have been added
reviews from page [5] have been added
reviews from page [6] have been added
reviews from page [7] have been added
reviews from page [8] have been added
reviews from page [9] have been added
reviews from page [10] have been added


['Absolute Masterpiece!!This movie will make a DC fan shed tears of joy coz its like a dream come true,every character is properly introduced with a touching backstory especially Cyborg,those who are not big fans of him will turn into Cyborg Superfans after watching this movie,apart from Cyborg a good introduction of Flash and his true capabilities,his contribution in the league and why he is one of the most important element of the group is clearly defined in this movie,action scenes are spine chilling and visually stuning like any other snyder movie and after every 5-10 mins there is some or the other action sequence which keeps the audience engaged and excited ,not even a single minute feels boring and the badass Steppenwolf is really really Frightening in this one a badass villain indeed completely opposite from what we saw in the 2017 Disaster called Josstice league.So yeah its a must watch,4 hrs will go by in a flash and at the end you will be left wanting for more!!',
 "I think 

In [11]:
classification_dataloader = modified_texts(reviews)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

Tesla T4


In [13]:
with open('/content/drive/MyDrive/Final_project/bert_model.pkl','rb') as f:
    model_ = pickle.load(f)

In [14]:
model_.eval()
test_preds = []

for batch in classification_dataloader:
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        logits = model_(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = logits[0].detach().cpu().numpy()

    batch_preds = np.argmax(logits, axis=1)
    test_preds.extend(batch_preds)

In [15]:
test_preds

[1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1]

In [16]:
positive = sum(1 for elem in test_preds if elem == 1)
positive

62

In [17]:
negative = sum(1 for elem in test_preds if elem == 0)
negative

38

In [18]:
general_text = []
if (positive >= negative):
  for i in range(len(test_preds)):
    if (test_preds[i] == 1):
      general_text.append(reviews[i])
else:
  for i in range(test_preds):
    if (test_preds[i] == 0):
      general_text.append(reviews[i])

general_text

['Absolute Masterpiece!!This movie will make a DC fan shed tears of joy coz its like a dream come true,every character is properly introduced with a touching backstory especially Cyborg,those who are not big fans of him will turn into Cyborg Superfans after watching this movie,apart from Cyborg a good introduction of Flash and his true capabilities,his contribution in the league and why he is one of the most important element of the group is clearly defined in this movie,action scenes are spine chilling and visually stuning like any other snyder movie and after every 5-10 mins there is some or the other action sequence which keeps the audience engaged and excited ,not even a single minute feels boring and the badass Steppenwolf is really really Frightening in this one a badass villain indeed completely opposite from what we saw in the 2017 Disaster called Josstice league.So yeah its a must watch,4 hrs will go by in a flash and at the end you will be left wanting for more!!',
 "I think 

In [19]:
general_text = "".join(general_text).replace('\n','')

general_text

'Absolute Masterpiece!!This movie will make a DC fan shed tears of joy coz its like a dream come true,every character is properly introduced with a touching backstory especially Cyborg,those who are not big fans of him will turn into Cyborg Superfans after watching this movie,apart from Cyborg a good introduction of Flash and his true capabilities,his contribution in the league and why he is one of the most important element of the group is clearly defined in this movie,action scenes are spine chilling and visually stuning like any other snyder movie and after every 5-10 mins there is some or the other action sequence which keeps the audience engaged and excited ,not even a single minute feels boring and the badass Steppenwolf is really really Frightening in this one a badass villain indeed completely opposite from what we saw in the 2017 Disaster called Josstice league.So yeah its a must watch,4 hrs will go by in a flash and at the end you will be left wanting for more!!I think it is 

In [20]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model.to(device)

article_input_ids = tokenizer.batch_encode_plus([general_text], return_tensors='pt', max_length=1024)['input_ids']
article_input_ids = article_input_ids.to(device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=512,
                             min_length=128,
                             no_repeat_ngram_size=5)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: ** '+summary_txt))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


> **Summary: ** The Snyder Cut was we needed when it first released but I'm glad they stayed on course. Painful to see what could have been (Batfleck, Henry Cavill, Cyborg), but glorious to see Snyder's vision for this film--if not for the extended universe--come to life. The personality of the characters have got their due. A four hour span provides that deserved depth that was missing in the initial release. A Justice League movie truly worthy of the title. A visually stunning, multilayered masterclass in cinema. A film that stands as proof that you can divert from the cookie cutter Superhero formula and deliver something unique and great.